In [229]:
%load_ext autoreload
%autoreload 2

import docker
import logging

from pymongo import MongoClient
from jtlutil.docker.manager import * 
from jtlutil.docker.db import * 
from jtlutil.config import get_config

config = get_config('./config/config.env')

mongo_client = MongoClient(config.MONGO_PUBLIC)

dcsr = DockerContainerStatsRepository(mongo_client)

from pydantic import Field
from faker import Faker

fake = Faker()

def create_fake_docker_container_stats() -> DockerContainerStats:
    return DockerContainerStats(
        container_id=fake.uuid4(),
        state=fake.word(),
        name=fake.word(),
        memory_usage=fake.random_number(digits=6),
        hostname=fake.hostname(),
        node=fake.word(),
        created=fake.date_time().isoformat()
    )
    
dcsr.delete_all()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [232]:
# Check that Sconstruction failes for docker uris ( rather than swarm ) 
try:
    dcsr = DockerManager.new(config.SSH_URI_DOCKER, mongo_client=mongo_client)
except AssertionError as e:
    pass
else:
    raise AssertionError("Expected an AssertionError when trying to create a new DockerManager with an invalid SSH URI")
    

In [236]:

# Generate and insert 10 fake Docker container stats into the repository
for _ in range(10):
    fake_stat = create_fake_docker_container_stats()
    dcsr.update(fake_stat)

print("Inserted 10 fake Docker container stats into the repository.")

N = len(dcsr.all)
print("N=", N)

for record in  dcsr.all:
    new_fake_stat = create_fake_docker_container_stats()
    new_fake_stat.container_id = record.container_id
    dcsr.update(new_fake_stat)

assert len(dcsr.all) == N, f"The number of records in the repository is not {N}"

for record in  dcsr.all:
    new_fake_stat = create_fake_docker_container_stats()
    new_fake_stat.container_id = record.container_id
    dcsr.update(new_fake_stat)

assert len(dcsr.all) == N, f"The number of records in the repository is not {N}"



Inserted 10 fake Docker container stats into the repository.
N= 10


In [237]:
dcsr.delete_all()

csm = DockerManager.new(config.SSH_URI_SWARM, mongo_client=mongo_client)


In [243]:
csm.collect_containers()

assert len(dcsr.all) == N, f"The number of records in the repository is not {N}"

dcsr.delete_all()

assert len(dcsr.all) == 0 